# Applying MLRun onto Existed Training Code

MLRun can be applied onto your training code quickly and easily, with a one-line solution providing many quality of
life features.

MLRun takes your training code and provide you insightful experiments tracking with auto-logging plots and tables,
manage your files with its artifacts and distribute your training using Dask and Horovod. Everything is done with
`mlrun.frameworks` and its `apply_mlrun` function.

After the integration with the code, simply create an MLRun function out of it and run like any other function.

## Applying MLRun

There are 2 options for you to integrate MLRun into your training code:

* You can manually use the `mlrun.frameworks` package.
* Or use our simple one-line solution function called `apply_mlrun`.

Both are highly customizable and provide quick and easy integration to all of MLRun's quality of life features.

### MLRun Frameworks

`mlrun.frameworks` is a sub-package in MLRun - a collection of commonly used machine and deep learning frameworks fully
supported and integrated in MLRun. Each framework supported will benefit many quality of life features we believe are
essential for a [healthy development environment]() like automatic logging, experiment tracking, resources, models and
files management, distributed training and hyperparameter tuning.

All the features mentioned are enabled for every framework by 4 main classes:

* **`MLRunInterface`** - An interface class providing MLRun's features by wrapping the framework's model and / or module
  to seamlessly enable them.
* **`ModelHandler`** - As its name suggests, it handles a framework's model and all of its related files. With 4 simple
  methods: `load`, `save`, `log` and `update` you are able to get or store your model of choice, enabling straight to
  the point and flexible file management. That includes module imports, custom objects (like python files of layers,
  optimizers and metrics) and everything else that is needed to load your model, all in a single class.
* **`ArtifactsLibrary`** - A static class of artifacts you can add to your framework's training to produce as outputs to
  further increase your insights and experiment tracking and comparisons.
* **`ModelServer`** - A serving class dedicated to the framework of choice, ready to be used in a realtime serving
  function with high performance and monitoring capabilities.
* Additional per framework classes (like a `MetricsLibrary` for SciKit-Learn).

Orchestrating on all the above is our shortcut function `apply_mlrun`.

In [ ]:
from mlrun.frameworks.sklearn import (
    SKLearnMLRunInterface,
    SKLearnModelHandler,
    SKLearnArtifactsLibrary,
    SklearnModelServer,
    SKLearnMetricsLibrary
)

### The `apply_mlrun` Function

The `apply_mlrun` function is MLRun's shortcut function to wrap your framework code and enable MLRun's quality of life
features with a single line of code! No need to change your training code in order to assign GPUs and / or distribute it. No need to keep track of parameters
and manually collect and plot metrics. `apply_mlrun` is taking care of it all.

The `apply_mlrun` function is framework specific and can be imported from MLRun's frameworks package, Here we will
import TensorFlow.Keras's function:

In [ ]:
from mlrun.frameworks.tf_keras import apply_mlrun

From here, call it before calling `fit` or `train`, depends on your framework of choice. To demonstrate how easy it is,
we will use `apply_mlrun` on top of TensorFlow's
[Training a neural network on MNIST with Keras](https://www.tensorflow.org/datasets/keras_example) example:

In [ ]:
...

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])

###########################################################################
# Applying MLRun on the created model to integrate with the training code:
apply_mlrun(model=model, model_name="mnist_classifier", tag="1.0.0")
###########################################################################

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

`apply_mlrun` can be used with its default settings, but it is highly flexible and rich with different options and
configurations. We highly suggest reading the docs of your favorite framework here in MLRun's docs and get to know the
additional tricks it can provide.

## Run Training

In order to run the training job all you need to do is create an MLRun Function out of it and run. Because you
integrated MLRun into your code, MLRun will take care of all the rest.

We will briefly showcase how to create and run a training job, to learn more read about [MLRun Functions]().

### Create the Training MLRun Function

After using `mlrun.frameworks` classes or simply `apply_mlrun` you can now create an MLRun function out of your training
code by using the function `mlrun.code_to_function` where you are able to select the image to use, additional
requirements and more.

Select the code's source (defaulted to the current python / notebook file) and you are good to go:

In [ ]:
import mlrun

mnist_trainer_function = mlrun.code_to_function(
    name="mnist_trainer",
    image=...,
    requirements=[...],
    kind="job"
)

One of the more important attributes in MLRun function creation is `kind`. For training, `kind` can be one of 3 options:

* **Job** (`"job"`) - For a single regular training run.
* **Open MPI** (`"mpijob"`) - For initializing Open MPI job. If `apply_mlrun` was used Horovod will be used
  automatically to distribute the training among the configured workers (for the DL frameworks TensorFlow and PyTorch).
* **Dask** (`"dask"`) - For initializing a Dask job. If `apply_mlrun` was used Dask will be used
  automatically to distribute the training among the configured workers (for the ML frameworks SciKit-Learn, XGBoost
  and LightGBM).

To know more about Dask and Horovod, visit the [distributed training]() page.

### Run

To run the training function, call the `run` method and thee results and artifacts will be visible in the MLRun UI.

To know more about the artifacts and results produced, visit the [experiment tracking and automatic logging]() page.

In [ ]:
mnist_trainer_run = mnist_trainer_function.run()

The run object returned will hold all the outputs of the training. You can review the results. see the artifacts and
pass them on as inputs to different functions as well.